# In this project I try to classify handwritten Persian numbers by MLP method. The data is freely available in: ‫‪https://users.encs.concordia.ca/~j_sadri/PersianDatabase.htm‬‬ .

### By using the following code I go through image files and resize thier size into 20*20 pixel and by using Image.open.convert I make a vector from each image

In [3]:
import numpy as np
from scipy.ndimage import convolve
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.externals import joblib
from sklearn.utils import shuffle
import os.path
from sklearn.metrics import confusion_matrix
from PIL import Image
import glob, os
from PIL import Image
from resizeimage import resizeimage

size = 20, 20

for filename in glob.glob("./HW2-PersianDigits-resized/Test/8/*.tif"):
    fd_img = open(filename, 'rb')
    img = Image.open(fd_img)
    img = resizeimage.resize_contain(img, [20, 20])
    img.save(filename, img.format)
    fd_img.close()

Featur_8_array = np.zeros(400)
for filename in glob.glob("./HW2-PersianDigits-resized/Test/8/*.tif"):
    fd_img = open(filename, 'rb')
    img = Image.open(fd_img).convert('L')
    arr = np.array(img)
    shape = arr.shape

    flat_arr = arr.ravel()

    vector = np.matrix(flat_arr)

    arr2 = np.asarray(vector).reshape(shape)

    img2 = Image.fromarray(arr2, 'L')
    Featur_8_array = np.vstack((Featur_8_array, flat_arr))


 ## Then by using the following code I make a dictionary from the Image vectors and stick label to vector of each number (same prosiger for Training and Test set)

In [78]:
test_dic = {'data':Featur_8_array, 'label': 8*np.ones(6)}
test_dic['label']
np.save('Featur_8_test.npy', test_dic) 


 # Then I merge all the produced dictionaries for Test and Training sets to the two following dictionary

In [10]:
read_dictionary0 = np.load('Featur_0.npy').item()
read_dictionary1 = np.load('Featur_1.npy').item()
read_dictionary2 = np.load('Featur_2.npy').item()
read_dictionary3 = np.load('Featur_3.npy').item()
read_dictionary4 = np.load('Featur_4.npy').item()
read_dictionary5 = np.load('Featur_5.npy').item()
read_dictionary6 = np.load('Featur_6.npy').item()
read_dictionary7 = np.load('Featur_7.npy').item()
read_dictionary8 = np.load('Featur_8.npy').item()
read_dictionary9 = np.load('Featur_9.npy').item()

dic_tot = {}
for key in read_dictionary0.keys():
    dic_tot[key] = np.vstack((read_dictionary0[key][1:], read_dictionary1[key][1:], read_dictionary2[key][1:] , read_dictionary3[key][1:]\
    ,read_dictionary4[key][1:],read_dictionary5[key][1:],read_dictionary6[key][1:],read_dictionary7[key][1:],read_dictionary8[key][1:],read_dictionary9[key][1:] ))



read_dictionary0_t = np.load('Featur_0_test.npy').item()
read_dictionary1_t = np.load('Featur_1_test.npy').item()
read_dictionary2_t = np.load('Featur_2_test.npy').item()
read_dictionary3_t = np.load('Featur_3_test.npy').item()
read_dictionary4_t = np.load('Featur_4_test.npy').item()
read_dictionary5_t = np.load('Featur_5_test.npy').item()
read_dictionary6_t = np.load('Featur_6_test.npy').item()
read_dictionary7_t = np.load('Featur_7_test.npy').item()
read_dictionary8_t = np.load('Featur_8_test.npy').item()
read_dictionary9_t = np.load('Featur_9_test.npy').item()

dic_tot_test = {}
for key in read_dictionary0_t.keys():
    dic_tot_test[key] = np.vstack((read_dictionary0_t[key][1:], read_dictionary1_t[key][1:], read_dictionary2_t[key][1:] , read_dictionary3_t[key][1:]\
    ,read_dictionary4_t[key][1:],read_dictionary5_t[key][1:],read_dictionary6_t[key][1:],read_dictionary7_t[key][1:],read_dictionary8_t[key][1:],read_dictionary9_t[key][1:] ))


 #  In the following part I train the system for  one layer with 200 neurons

In [30]:


PATH = 'mlp_model_200'

if __name__ == '__main__':
    
    X, y = dic_tot['data'], dic_tot['label'].ravel()
    X2, y2 = shuffle(X, y, random_state=0)
    
    X_train, X_test, y_train, y_test = train_test_split(X2 / 255., y2, test_size=0.25)

    print('Got MNIST with %d training- and %d test samples' % (len(y_train), len(y_test)))
    #print('Digit distribution in whole dataset:', np.bincount(y2.astype('int32')))

    clf = None
    if os.path.exists(PATH):
        print('Loading model from file.')
        clf = joblib.load(PATH).best_estimator_
    else:
        print('Training model.')
        # params = {'hidden_layer_sizes': [(256,), (512,), (128, 256, 128,)]}
        params = {'hidden_layer_sizes': [(200,)]}
        mlp = MLPClassifier(verbose=10, momentum=0.9, learning_rate='adaptive', activation='relu')
        clf = GridSearchCV(mlp, params, verbose=10, n_jobs=-1, cv=5)
        clf.fit(X_train, y_train)
        print('Finished with grid search with best mean cross-validated score:', clf.best_score_)
        # print('Best params appeared to be', clf.best_params_)
        joblib.dump(clf, PATH)
        clf = clf.best_estimator_

    print('Test accuracy:', clf.score(X_test, y_test))
    


Got MNIST with 150 training- and 50 test samples
Loading model from file.
Test accuracy: 0.24


 # After traing the system I calculate the confusion matrix and accuracy of the Test set.

In [31]:
X, y = dic_tot_test['data'], dic_tot_test['label'].ravel()
X, y = shuffle(X, y, random_state=0)
X_test=X 
y_test=y 

print('\n The confusion matrix is:\n' ,confusion_matrix(y_test,prediction))
print( '\n The accuracy is:', clf.score(X_test,y_test))


 The confusion matrix is:
 [[5 0 0 0 0 0 0 0 0 0]
 [0 5 0 0 0 0 0 0 0 0]
 [0 0 4 1 0 0 0 0 0 0]
 [0 0 0 4 1 0 0 0 0 0]
 [1 0 1 0 3 0 0 0 0 0]
 [0 0 0 0 0 5 0 0 0 0]
 [1 0 0 1 0 0 2 1 0 0]
 [0 0 0 0 0 0 0 5 0 0]
 [0 0 0 0 0 2 0 0 3 0]
 [0 0 0 0 0 0 0 0 0 5]]

 The accuracy is: 0.16


 # One layer with 600 neurons

In [35]:


PATH = 'mlp_model_600'

if __name__ == '__main__':
    
    X, y = dic_tot['data'], dic_tot['label'].ravel()
    X2, y2 = shuffle(X, y, random_state=0)
    
    X_train, X_test, y_train, y_test = train_test_split(X2 / 255., y2, test_size=0.25)

    print('Got MNIST with %d training- and %d test samples' % (len(y_train), len(y_test)))
    #print('Digit distribution in whole dataset:', np.bincount(y2.astype('int32')))

    clf = None
    if os.path.exists(PATH):
        print('Loading model from file.')
        clf = joblib.load(PATH).best_estimator_
    else:
        print('Training model.')
        # params = {'hidden_layer_sizes': [(256,), (512,), (128, 256, 128,)]}
        params = {'hidden_layer_sizes': [(600,)]}
        mlp = MLPClassifier(verbose=10, momentum=0.9, learning_rate='adaptive', activation='relu')
        clf = GridSearchCV(mlp, params, verbose=10, n_jobs=-1, cv=5)
        clf.fit(X_train, y_train)
        print('Finished with grid search with best mean cross-validated score:', clf.best_score_)
        # print('Best params appeared to be', clf.best_params_)
        joblib.dump(clf, PATH)
        clf = clf.best_estimator_

    print('Test accuracy:', clf.score(X_test, y_test))
    


Got MNIST with 150 training- and 50 test samples
Loading model from file.
Test accuracy: 0.94


In [36]:
X, y = dic_tot_test['data'], dic_tot_test['label'].ravel()
X, y = shuffle(X, y, random_state=0)
X_test=X 
y_test=y 

print('\n The confusion matrix is:\n' ,confusion_matrix(y_test,prediction))
print( '\n The accuracy is:', clf.score(X_test,y_test))


 The confusion matrix is:
 [[5 0 0 0 0 0 0 0 0 0]
 [0 5 0 0 0 0 0 0 0 0]
 [0 0 4 1 0 0 0 0 0 0]
 [0 0 0 4 1 0 0 0 0 0]
 [1 0 1 0 3 0 0 0 0 0]
 [0 0 0 0 0 5 0 0 0 0]
 [1 0 0 1 0 0 2 1 0 0]
 [0 0 0 0 0 0 0 5 0 0]
 [0 0 0 0 0 2 0 0 3 0]
 [0 0 0 0 0 0 0 0 0 5]]

 The accuracy is: 0.68


 # Two layer with 500 and 300 neurons

In [75]:


PATH = 'mlp_model_2Layer'

if __name__ == '__main__':
    
    X, y = dic_tot['data'], dic_tot['label'].ravel()
    X2, y2 = shuffle(X, y, random_state=0)
    
    X_train, X_test, y_train, y_test = train_test_split(X2 / 255., y2, test_size=0.25)

    print('Got MNIST with %d training- and %d test samples' % (len(y_train), len(y_test)))
    #print('Digit distribution in whole dataset:', np.bincount(y2.astype('int32')))

    clf = None
    if os.path.exists(PATH):
        print('Loading model from file.')
        clf = joblib.load(PATH).best_estimator_
    else:
        print('Training model.')
        # params = {'hidden_layer_sizes': [(256,), (512,), (128, 256, 128,)]}
        params = {'hidden_layer_sizes': [(500,300)]}
        mlp = MLPClassifier(verbose=10, momentum=0.9, learning_rate='adaptive', activation='relu')
        clf = GridSearchCV(mlp, params, verbose=10, n_jobs=-1, cv=5)
        clf.fit(X_train, y_train)
        print('Finished with grid search with best mean cross-validated score:', clf.best_score_)
        # print('Best params appeared to be', clf.best_params_)
        joblib.dump(clf, PATH)
        clf = clf.best_estimator_

    print('Test accuracy:', clf.score(X_test, y_test))
    


Got MNIST with 150 training- and 50 test samples
Loading model from file.
Test accuracy: 0.92


In [76]:
X, y = dic_tot_test['data'], dic_tot_test['label'].ravel()
X, y = shuffle(X, y, random_state=0)
X_test=X 
y_test=y 

print('\n The confusion matrix is:\n' ,confusion_matrix(y_test,prediction))
print( '\n The accuracy is:', clf.score(X_test,y_test))


 The confusion matrix is:
 [[5 0 0 0 0 0 0 0 0 0]
 [0 5 0 0 0 0 0 0 0 0]
 [0 0 4 1 0 0 0 0 0 0]
 [0 0 0 4 1 0 0 0 0 0]
 [1 0 1 0 3 0 0 0 0 0]
 [0 0 0 0 0 5 0 0 0 0]
 [1 0 0 1 0 0 2 1 0 0]
 [0 0 0 0 0 0 0 5 0 0]
 [0 0 0 0 0 2 0 0 3 0]
 [0 0 0 0 0 0 0 0 0 5]]

 The accuracy is: 0.74


 # One layer with 50 neurons


In [51]:


PATH = 'mlp_model_50'

if __name__ == '__main__':
    
    X, y = dic_tot['data'], dic_tot['label'].ravel()
    X2, y2 = shuffle(X, y, random_state=0)
    
    X_train, X_test, y_train, y_test = train_test_split(X2 / 255., y2, test_size=0.25)

    print('Got MNIST with %d training- and %d test samples' % (len(y_train), len(y_test)))
    #print('Digit distribution in whole dataset:', np.bincount(y2.astype('int32')))

    clf = None
    if os.path.exists(PATH):
        print('Loading model from file.')
        clf = joblib.load(PATH).best_estimator_
    else:
        print('Training model.')
        # params = {'hidden_layer_sizes': [(256,), (512,), (128, 256, 128,)]}
        params = {'hidden_layer_sizes': [(50,)]}
        mlp = MLPClassifier(verbose=10, momentum=0.9, learning_rate='adaptive', activation='relu')
        clf = GridSearchCV(mlp, params, verbose=10, n_jobs=-1, cv=5)
        clf.fit(X_train, y_train)
        print('Finished with grid search with best mean cross-validated score:', clf.best_score_)
        # print('Best params appeared to be', clf.best_params_)
        joblib.dump(clf, PATH)
        clf = clf.best_estimator_

    print('Test accuracy:', clf.score(X_test, y_test))
    


Got MNIST with 150 training- and 50 test samples
Loading model from file.
Test accuracy: 0.9


In [52]:
X, y = dic_tot_test['data'], dic_tot_test['label'].ravel()
X, y = shuffle(X, y, random_state=0)
X_test=X 
y_test=y 

print('\n The confusion matrix is:\n' ,confusion_matrix(y_test,prediction))
print( '\n The accuracy is:', clf.score(X_test,y_test))


 The confusion matrix is:
 [[5 0 0 0 0 0 0 0 0 0]
 [0 5 0 0 0 0 0 0 0 0]
 [0 0 4 1 0 0 0 0 0 0]
 [0 0 0 4 1 0 0 0 0 0]
 [1 0 1 0 3 0 0 0 0 0]
 [0 0 0 0 0 5 0 0 0 0]
 [1 0 0 1 0 0 2 1 0 0]
 [0 0 0 0 0 0 0 5 0 0]
 [0 0 0 0 0 2 0 0 3 0]
 [0 0 0 0 0 0 0 0 0 5]]

 The accuracy is: 0.7


 # Two layer with 50 and 25 neurons

In [65]:


PATH = 'mlp_model_2Layer_2'

if __name__ == '__main__':
    
    X, y = dic_tot['data'], dic_tot['label'].ravel()
    X2, y2 = shuffle(X, y, random_state=0)
    
    X_train, X_test, y_train, y_test = train_test_split(X2 / 255., y2, test_size=0.25)

    print('Got MNIST with %d training- and %d test samples' % (len(y_train), len(y_test)))
    #print('Digit distribution in whole dataset:', np.bincount(y2.astype('int32')))

    clf = None
    if os.path.exists(PATH):
        print('Loading model from file.')
        clf = joblib.load(PATH).best_estimator_
    else:
        print('Training model.')
        # params = {'hidden_layer_sizes': [(256,), (512,), (128, 256, 128,)]}
        params = {'hidden_layer_sizes': [(50,25)]}
        mlp = MLPClassifier(verbose=10, momentum=0.9, learning_rate='adaptive', activation='relu')
        clf = GridSearchCV(mlp, params, verbose=10, n_jobs=-1, cv=5)
        clf.fit(X_train, y_train)
        print('Finished with grid search with best mean cross-validated score:', clf.best_score_)
        # print('Best params appeared to be', clf.best_params_)
        joblib.dump(clf, PATH)
        clf = clf.best_estimator_

    print('Test accuracy:', clf.score(X_test, y_test))
    


Got MNIST with 150 training- and 50 test samples
Loading model from file.
Test accuracy: 0.94


In [67]:
X, y = dic_tot_test['data'], dic_tot_test['label'].ravel()
X, y = shuffle(X, y, random_state=0)
X_test=X 
y_test=y 

print('\n The confusion matrix is:\n' ,confusion_matrix(y_test,prediction))
print( '\n The accuracy is:', clf.score(X_test,y_test))


 The confusion matrix is:
 [[5 0 0 0 0 0 0 0 0 0]
 [0 5 0 0 0 0 0 0 0 0]
 [0 0 4 1 0 0 0 0 0 0]
 [0 0 0 4 1 0 0 0 0 0]
 [1 0 1 0 3 0 0 0 0 0]
 [0 0 0 0 0 5 0 0 0 0]
 [1 0 0 1 0 0 2 1 0 0]
 [0 0 0 0 0 0 0 5 0 0]
 [0 0 0 0 0 2 0 0 3 0]
 [0 0 0 0 0 0 0 0 0 5]]

 The accuracy is: 0.7


# I have imported the following cell to show how to load a previously learned model and use it

In [15]:


from sklearn.externals import joblib
import numpy as np
import numpy as np
from scipy.ndimage import convolve
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.externals import joblib
from sklearn.utils import shuffle
import os.path
from sklearn.metrics import confusion_matrix
from PIL import Image
import glob, os
from PIL import Image
from resizeimage import resizeimage
PATH = 'mlp_model_2Layer_2'
clf = joblib.load(PATH).best_estimator_

read_dictionary0 = np.load('Featur_0.npy').item()
read_dictionary1 = np.load('Featur_1.npy').item()
read_dictionary2 = np.load('Featur_2.npy').item()
read_dictionary3 = np.load('Featur_3.npy').item()
read_dictionary4 = np.load('Featur_4.npy').item()
read_dictionary5 = np.load('Featur_5.npy').item()
read_dictionary6 = np.load('Featur_6.npy').item()
read_dictionary7 = np.load('Featur_7.npy').item()
read_dictionary8 = np.load('Featur_8.npy').item()
read_dictionary9 = np.load('Featur_9.npy').item()

dic_tot = {}
for key in read_dictionary0.keys():
    dic_tot[key] = np.vstack((read_dictionary0[key][1:], read_dictionary1[key][1:], read_dictionary2[key][1:] , read_dictionary3[key][1:]\
    ,read_dictionary4[key][1:],read_dictionary5[key][1:],read_dictionary6[key][1:],read_dictionary7[key][1:],read_dictionary8[key][1:],read_dictionary9[key][1:] ))



read_dictionary0_t = np.load('Featur_0_test.npy').item()
read_dictionary1_t = np.load('Featur_1_test.npy').item()
read_dictionary2_t = np.load('Featur_2_test.npy').item()
read_dictionary3_t = np.load('Featur_3_test.npy').item()
read_dictionary4_t = np.load('Featur_4_test.npy').item()
read_dictionary5_t = np.load('Featur_5_test.npy').item()
read_dictionary6_t = np.load('Featur_6_test.npy').item()
read_dictionary7_t = np.load('Featur_7_test.npy').item()
read_dictionary8_t = np.load('Featur_8_test.npy').item()
read_dictionary9_t = np.load('Featur_9_test.npy').item()

dic_tot_test = {}
for key in read_dictionary0_t.keys():
    dic_tot_test[key] = np.vstack((read_dictionary0_t[key][1:], read_dictionary1_t[key][1:], read_dictionary2_t[key][1:] , read_dictionary3_t[key][1:]\
    ,read_dictionary4_t[key][1:],read_dictionary5_t[key][1:],read_dictionary6_t[key][1:],read_dictionary7_t[key][1:],read_dictionary8_t[key][1:],read_dictionary9_t[key][1:] ))

X, y = dic_tot_test['data'], dic_tot_test['label'].ravel()
X2, y2 = shuffle(X, y, random_state=0)

    
X_train, X_test, y_train, y_test = train_test_split(X2 / 255., y2, test_size=0.25)

clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
print('\n The confusion matrix is:\n' ,confusion_matrix(y_test,prediction))
print( '\n The accuracy is:', clf.score(X_test,y_test))

Iteration 1, loss = 2.52443136
Iteration 2, loss = 2.34559998
Iteration 3, loss = 2.27130026
Iteration 4, loss = 2.27782508
Iteration 5, loss = 2.27198893
Iteration 6, loss = 2.24670804
Iteration 7, loss = 2.20838077
Iteration 8, loss = 2.18289159
Iteration 9, loss = 2.17751861
Iteration 10, loss = 2.16989941
Iteration 11, loss = 2.15630954
Iteration 12, loss = 2.13848831
Iteration 13, loss = 2.12122351
Iteration 14, loss = 2.10543111
Iteration 15, loss = 2.09079174
Iteration 16, loss = 2.07657041
Iteration 17, loss = 2.06215777
Iteration 18, loss = 2.04713199
Iteration 19, loss = 2.03294542
Iteration 20, loss = 2.01879449
Iteration 21, loss = 2.00392343
Iteration 22, loss = 1.99160373
Iteration 23, loss = 1.97546755
Iteration 24, loss = 1.95919945
Iteration 25, loss = 1.94431207
Iteration 26, loss = 1.92674342
Iteration 27, loss = 1.90997489
Iteration 28, loss = 1.89606094
Iteration 29, loss = 1.88147772
Iteration 30, loss = 1.86413279
Iteration 31, loss = 1.84792082
Iteration 32, los

/home/zhoolideh.m/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
